In [9]:
import numpy as np
import numpy as np
import subprocess
import time
import copy
from utils.opt_utils import *
import os
import h5py
import pickle
from qutip import *
import utils.pulse_configs as pc
import time
from exp_ctrl_vqe_cost_function_pi import exp_g1_cost_func_xt
from scipy.optimize import minimize, basinhopping, brute

In [10]:
config_dict = pc.config_dict

In [11]:
# Defining relevant values
q_ch = config_dict["qubit"]["ch"] 
A_ch = config_dict["storage_A"]["ch"]
B_ch = config_dict["storage_B"]["ch"]

wq = config_dict["qubit"]["selective_freqs"]["g"]["ge"]
wa_A = config_dict["storage_A"]["freq"]
wa_B = config_dict["storage_B"]["freq"]

selective_pi_sigma = config_dict["qubit"]["ge"]["selective_sigma"]
selective_pi_amp = config_dict["qubit"]["ge"]["selective_pi_gain"]

In [12]:
max_segs = 20
us = 0.000001
time_start = 0 * us
time_stop = 2 * us
init_amp = 1000

N_pts = 22

num_drives = 2
chs = [2, 4]
drive_freqs = [wq, wa_A]
verbose = False
plot_opt_pulses = False
plot_pulse_viewer = False

drive_freqs = np.array(drive_freqs)

client_args = [chs, drive_freqs, verbose, plot_opt_pulses, plot_pulse_viewer, time_start, time_stop]

In [13]:
def snap_to_cost_eval(opts, *args):
    args = args[0]
    d_amp_1_r = opts[0]
    d_amp_2_r = opts[1]
    d_amp_2_i = opts[2]
    d_time_2 = opts[3]
    
    print(f'Trial parameters: {opts}')
    
    time_start = args[5]
    time_stop = args[6]
    
    qubit_time = (time_start + time_stop) / 2
    qubit_amp = 2 * selective_pi_amp
    
    qubit_func = gauss_gen(qubit_amp, qubit_time, selective_pi_sigma * us, 1)
    
    d_1_func_r = gauss_gen(d_amp_1_r, 0.2 * us, 0.05 * us, 1)
    d_2_func_r = gauss_gen(d_amp_2_r, d_time_2 * us, 0.05 * us, 1)
    d_2_func_i = gauss_gen(d_amp_2_i, d_time_2 * us, 0.05 * us, 1)
    
    drive_amp_funcs = [[qubit_func],
                       [zero],
                       [d_1_func_r, d_2_func_r],
                       [d_2_func_i]]
    
    comp_amp_funcs = [func_sum(np.array(drive_amp_funcs[i])) for i in range(len(drive_amp_funcs))]
    
    t_arr = np.linspace(time_start, time_stop, N_pts)[1:-1]
    
    amp_pts = np.array([[(comp_amp_funcs[i])(t_arr)] for i in range(len(comp_amp_funcs))])
    
    t_pts = np.array([t_arr for i in range(len(amp_pts))])
    
    opts = np.append(amp_pts.flatten(), t_pts.flatten())
    
    return exp_g1_cost_func_xt(opts, *args)

In [14]:
verbose = False

x0=[14000, 7000, 1000, 1.8]

bounds = [[-32000, 32000],
          [-32000, 32000],
          [-32000, 32000],
          [1.5, 1.9]]

res = minimize(snap_to_cost_eval, x0=x0, method='Nelder-Mead', args=client_args, options={"maxiter": 1000}, bounds=bounds)

C:\Users\Wang_Lab\anaconda3\envs\qcrl-server\lib\site-packages\scipy\optimize\_minimize.py:521: RuntimeWarning: Method Nelder-Mead cannot handle constraints nor bounds.
  warn('Method %s cannot handle constraints nor bounds.' % method,


Trial parameters: [1.4e+04 7.0e+03 1.0e+03 1.8e+00]
Trial I time 0: [0.00000000e+00 9.52380952e-08 1.90476190e-07 2.85714286e-07
 3.80952381e-07 4.76190476e-07 5.71428571e-07 6.66666667e-07
 7.61904762e-07 8.57142857e-07 9.52380952e-07 1.04761905e-06
 1.14285714e-06 1.23809524e-06 1.33333333e-06 1.42857143e-06
 1.52380952e-06 1.61904762e-06 1.71428571e-06 1.80952381e-06
 1.90476190e-06 1.99999999e-06]
Trial I amp 0: [  0. 184. 255. 341. 439. 545. 653. 755. 842. 905. 939. 939. 905. 842.
 755. 653. 545. 439. 341. 255. 184.   0.]
Trial Q time 0: [0.00000000e+00 9.52380952e-08 1.90476190e-07 2.85714286e-07
 3.80952381e-07 4.76190476e-07 5.71428571e-07 6.66666667e-07
 7.61904762e-07 8.57142857e-07 9.52380952e-07 1.04761905e-06
 1.14285714e-06 1.23809524e-06 1.33333333e-06 1.42857143e-06
 1.52380952e-06 1.61904762e-06 1.71428571e-06 1.80952381e-06
 1.90476190e-06 1.99999999e-06]
Trial Q amp 0: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Trial I time 1: [0.00000000e+00


KeyboardInterrupt

